# Exponential smoothing

Zbiór danych: https://www.kaggle.com/datasets/hamidrezabakhtaki/fmcg-export-sale

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing

In [2]:
# puść ten kod, 
# jeżeli wywołujesz plik  w folderze rozwiąznaia, 
# a ramka danych znajduje się w folderze data
import os 
os.chdir('../')

In [ ]:
df_resampled = pd.read_csv('data\sales_monthly.csv',index_col=0)
df_resampled.sort_index(inplace=True)
df_resampled.index = pd.to_datetime(df_resampled.index)
df_resampled.index.freq = 'MS'

In [6]:
# Proste wygładzanie wykładnicze 
ses_model =SimpleExpSmoothing(df_resampled['QTY']).fit(smoothing_level=0.5,optimized= False)
df_resampled['ses_forecast'] = ses_model.fittedvalues

In [ ]:
df_resampled.head()

In [ ]:
# Wygładzanie z trendem 
holt_model = ExponentialSmoothing(df_resampled['QTY'],trend='add',seasonal=None).fit(optimized = True)
df_resampled['holt_forecast'] = holt_model.fittedvalues

In [ ]:
#Wygładzanie z sezonowością i trendem (Holt-Winters) 
hw_model = ExponentialSmoothing(df_resampled['QTY'],trend='add',seasonal='add', seasonal_periods=12).fit(optimized = True)
df_resampled['hw_foecast'] = holt_model.fittedvalues

In [12]:
# liczba przyszłych predykcji
future_periods = 12

In [14]:
# Prognoza dla każdego modelu
ses_forecast = pd.DataFrame(ses_model.forecast(future_periods),columns =['ses_forecast'])
holt_forecast = pd.DataFrame(holt_model.forecast(future_periods),columns =['holt_forecast'])
hw_forecast = pd.DataFrame(hw_model.forecast(future_periods),columns = ['hw_foecast'])

In [ ]:
hw_forecast

In [ ]:
# Połączenie różnych forecastów
all_forecasts = ses_forecast.join(holt_forecast).join(hw_forecast)
all_forecasts.head()

In [20]:
# Połączenie okresu historycznego z forecastem
df_total = pd.concat([df_resampled,all_forecasts])

In [ ]:
df_total.head()

In [22]:
del df_total['holt_foecast']

In [ ]:
df_total.head()

In [24]:
del df_total['Total Price']

In [ ]:
# Wizualizacja
df_total.plot(figsize=(12,6))
plt.show()